# step1 download your ResNet caffemodel

# step2 download its deploy.prototxt

# step3 preparing data 

### now my data structure is here

#### it is a binary classifycation problem , so i put all my data in two folder

In [1]:
import os
os.chdir(r"/xuan/counter")
os.listdir("./data")

['iron_neg', 'iron_pos']

#### first of all , split your data into trainset and testset

- trainset
    - iron_neg
    - iron_pos
- testset
    - iron_neg
    - iron_pos

In [4]:
import glob,shutil,os,sys
import random
if os.path.exists("./data/testset"):
    shutil.rmtree("./data/testset")
os.mkdir("./data/testset")
if os.path.exists("./data/trainset"):
    shutil.rmtree("./data/trainset")
os.mkdir("./data/trainset")

testSetNum = 100

folderList = glob.glob("./data/iron*/")
folderList
for folder in folderList:
    classes = folder.split("/")[2]
    if os.path.exists("./data/testset/"+classes):
        shutil.rmtree("./data/testset/"+classes)
    os.mkdir("./data/testset/"+classes)
    
    if os.path.exists("./data/trainset/"+classes):
        shutil.rmtree("./data/trainset/"+classes)
    os.mkdir("./data/trainset/"+classes)
    
    imgList = os.listdir(folder)
    random.shuffle(imgList)
    
    train = imgList[:-testSetNum]
    test  = imgList[-testSetNum:]
    

    for file in train:
        shutil.copy(folder +  file , './data/trainset/' + classes + '/'+file  )

    for file in test:
        shutil.copy(folder +  file , './data/testset/' + classes + '/'+file  )

    
    

In [5]:
classDict = [i for i in folderList] 

In [7]:
classDict = {i.split('/')[2]:idx for idx,i in enumerate(folderList)}
classDict

{'iron_neg': 0, 'iron_pos': 1}

#### arrange to data to use caffe ImageData layer

- generate a train.txt which contains the image info and class info
```
    mnist/train/5/00000.png 5
    mnist/train/0/00001.png 0
    mnist/train/4/00002.png 4
    mnist/train/1/00003.png 1
    mnist/train/9/00004.png 9
    mnist/train/2/00005.png 2
    mnist/train/1/00006.png 1
```

In [18]:
import glob
import pandas as pd
trainfile = [ (data[2:], classDict[data.split('/')[3]]  ) for data in glob.iglob('./data/trainset/**/*')]
trainfile = pd.DataFrame(trainfile)
trainfile.to_csv("./data/train.txt",  sep = ' ',header = None , index =False)


testfile = [ (data[2:], classDict[data.split('/')[3]]  ) for data in glob.iglob('./data/testset/**/*')]
testfile = pd.DataFrame(testfile)
testfile.to_csv("./data/test.txt",  sep = ' ',header = None , index =False)

#### check the caffemodel place


In [20]:
caffemodel  = './model/ResNet-50-model.caffemodel'
deployProto = './model/deploy.prototxt' 



### build a caffe model to first check caffe network

In [22]:
import caffe
net = caffe.Net(deployProto,caffemodel,caffe.TEST)

In [9]:
import cPickle as pickle

In [37]:
data = pickle.load(open('./model/R-50.pkl','rb'))
[i for i in data.keys() if i.split('_')[0] == 'res2'][:20]

[u'res2_0_branch2a_bn_s',
 u'res2_1_branch2a_bn_s',
 u'res2_2_branch2a_w',
 u'res2_1_branch2c_b',
 u'res2_0_branch2a_bn_b',
 u'res2_1_branch2a_bn_b',
 u'res2_1_branch2c_w',
 u'res2_2_branch2a_b',
 u'res2_1_branch2c_bn_b',
 u'res2_0_branch2b_bn_b',
 u'res2_0_branch2c_bn_s',
 u'res2_1_branch2c_bn_s',
 u'res2_0_branch2c_bn_b',
 u'res2_0_branch2a_w',
 u'res2_0_branch1_b',
 u'res2_2_branch2a_bn_b',
 u'res2_1_branch2b_b',
 u'res2_0_branch1_w',
 u'res2_2_branch2a_bn_s',
 u'res2_0_branch2c_w']

In [39]:
[i for i in net.blobs.keys() if i[:4] == 'res2']

['res2a_branch1',
 'res2a_branch2a',
 'res2a_branch2b',
 'res2a_branch2c',
 'res2a',
 'res2a_res2a_relu_0_split_0',
 'res2a_res2a_relu_0_split_1',
 'res2b_branch2a',
 'res2b_branch2b',
 'res2b_branch2c',
 'res2b',
 'res2b_res2b_relu_0_split_0',
 'res2b_res2b_relu_0_split_1',
 'res2c_branch2a',
 'res2c_branch2b',
 'res2c_branch2c',
 'res2c',
 'res2c_res2c_relu_0_split_0',
 'res2c_res2c_relu_0_split_1']

### setup your train.prototxt

### 需要进行这样的修改

- 把 数据输入层修改为

```
name: "ResNet-50"
layer {
  name: "data"
  type: "ImageData"
  top: "data"
  top: "label"
  include {
    phase: TRAIN
  }
  transform_param {
    mirror: true
    crop_size: 224
    mean_value: 103.94
    mean_value: 116.78
    mean_value: 123.68
    scale: 0.017
  }
  image_data_param {
    source: "C:/Users/an/Documents/study/counter/data/train.txt"
    root_folder: "C:/Users/an/Documents/study/counter/"
    new_height: 224
    new_width: 224
    batch_size: 10
    rand_skip: 100
    shuffle: true
  }
}


layer {
  name: "data"
  type: "ImageData"
  top: "data"
  top: "label"
  include {
    phase: TEST
  }
  transform_param {
    mirror: true
    crop_size: 224
    mean_value: 103.94
    mean_value: 116.78
    mean_value: 123.68
    scale: 0.017
  }
  image_data_param {
    source: "C:/Users/an/Documents/study/counter/data/test.txt"
    root_folder: "C:/Users/an/Documents/study/counter/"
    new_height: 224
    new_width: 224
    batch_size: 10
    rand_skip: 100
    shuffle: true
  }
}


```

- 把output 修改成



### train your model 

In [ ]:
!caffe train -solver /xuan/counter/model/solver.prototxt -gpu 2 -weights /xuan/counter/model/ResNet-50-model.caffemodel    2>&1 | tee -a /xuan/counter/model/my_model.log 
    

### 从log中绘制loss

In [4]:
!python /opt/caffe/tools/extra/parse_log.py \
     /xuan/counter/model/my_model.log \
     /xuan/counter/model/

### 可以看出20000步的model是最佳的